In [1]:
import pandas as pd
import glob
import numpy as np

# 1. 必要なライブラリをインポートする
from pathlib import Path

# 2. "_Word_list.csv"ファイルを読み込む
file_path = 'output/Value/Word_list'  # フォルダのパスを指定してください
word_list_files = glob.glob(f"{file_path}/*_Word_list.csv")

# Keyword1.csvを読み込む
keyword_df = pd.read_csv('output/Value/Keywords1.csv', header=None)
keywords = keyword_df[0].tolist()

results = []
histogram_results = []

# 3. 各ファイルについて、"件数"に基づいて順位を計算する
for file in word_list_files:
    df = pd.read_csv(file)
    
    # 4. "件数"の順位を計算
    df['順位'] = df['件数'].rank(method='min', ascending=False).astype(int)
    
    # 5. Keyword1.csvの単語と照合し、該当するレコードを抽出
    matched_df = df[df['単語'].isin(keywords)]
    
    # 結果をリストに追加
    results.append(matched_df)
    
    # ヒストグラムを計算するための準備
    industry_code = matched_df['業種'].iloc[0]
    stock_code = matched_df['コード'].iloc[0]
    stock_name = matched_df['銘柄名'].iloc[0]
    analysis_range = matched_df['分析範囲'].iloc[0]
    
    # ヒストグラム計算（幅100）
    bin_width = 100
    max_rank = matched_df['順位'].max()
    bins = np.arange(1, max_rank + bin_width, bin_width)
    histogram, bin_edges = np.histogram(matched_df['順位'], bins=bins)
    
    # ヒストグラム結果をデータフレームに変換
    for i in range(len(histogram)):
        bin_range = f"{bin_edges[i]}-{bin_edges[i+1]-1}"
        count = histogram[i]
        histogram_results.append({
            '業種': industry_code,
            'コード': stock_code,
            '銘柄名': stock_name,
            '分析範囲': analysis_range,
            '順位範囲': bin_range,
            '件数': count
        })

# 6. 全ての結果を一つのデータフレームにまとめる
final_df = pd.concat(results)

# 結果をCSVに出力
final_df.to_csv('output/Value/aggregated_rank_results_value.csv', index=False, encoding='utf-8-sig')

# ヒストグラム結果をデータフレームにまとめる
histogram_df = pd.DataFrame(histogram_results)

# ヒストグラム結果をCSVに出力
histogram_df.to_csv('output/Value/histogram_results_value.csv', index=False, encoding='utf-8-sig')

print("処理が完了しました。結果はaggregated_rank_results_value.csvとhistogram_results_value.csvに保存されました。")


処理が完了しました。結果はaggregated_rank_results_value.csvとhistogram_results_value.csvに保存されました。
